In [1]:
from dataReadFromDisk import *

In [2]:
df = load_df_from_csv()

DataFrame loaded from: E:\\Notebooks\\osu\processed-metadata\processed-metadata.csv


In [3]:
process_audio_and_beatmap_for_model_all_and_save(df)

ValueError: PARSING ERROR SOMEWHERE : Beatmap timestamp 10081.0ms is smaller than previous which is10360.0 10000.0 9640.0in map 6fc07475587f45a69547eb7f9b51a0c3/0013ddfd8bd55fdccc0b253e313b7a60.osu

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed

# Define the model
def create_lstm_model(input_shape, output_units):
    model = Sequential()
    
    # LSTM layer
    model.add(LSTM(units=128, return_sequences=True, input_shape=input_shape))
    
    # TimeDistributed layer with Dense to apply Dense to each time step
    model.add(TimeDistributed(Dense(units=output_units, activation='softmax')))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Assuming the input and output arrays are stored in .npy files
# Example function to load data
def load_data(input_files, output_files):
    X, Y = [], []
    
    for input_file, output_file in zip(input_files, output_files):
        X.append(np.load(input_file))
        Y.append(np.load(output_file))
    
    X = np.concatenate(X, axis=0)  # Concatenate all chunks into one large array
    Y = np.concatenate(Y, axis=0)
    
    return X, Y

# Example file lists
input_files = ["input_chunk1.npy"]
output_files = ["output_chunk1.npy"]

# # Load the data
# X, Y = load_data(input_files, output_files)

# # Define input shape (timesteps, features) and number of output units
# input_shape = (X.shape[1], X.shape[2])  # (timesteps, features)
# output_units = Y.shape[2]  # Number of features in the output sequence

# # Create the model
# model = create_lstm_model(input_shape, output_units)

# # Train the model
# model.fit(X, Y, batch_size=32, epochs=20)

# # Save the trained model
# model.save('osu_beatmap_lstm_model.h5')
